In [ ]:
#Install muna ito
!pip install ultralytics --user

In [2]:
#then ito
#dapat nvidia gpu and version 12.9 ang cuda
#adjust code kung iba version ng cuda
#kapag hindi nvidia gpu, i-gpt na lang kung paano ilipat from GPU to CPU
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version used by PyTorch: {torch.version.cuda}")
print(f"Number of GPUs: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    print(f"Current GPU: {torch.cuda.get_device_name(0)}")
    
    # Test CUDA
    x = torch.rand(5, 3)
    print(f"\nTensor on CPU: {x}")
    
    x = x.cuda()
    print(f"Tensor on GPU: {x}")
    print(f"Is tensor on CUDA? {x.is_cuda}")

PyTorch version: 2.5.1
CUDA available: False
CUDA version used by PyTorch: None
Number of GPUs: 0


In [3]:
pip install filterpy

Note: you may need to restart the kernel to use updated packages.


In [6]:
#kapag okay na sa setup and install, I-run muna ito para maload ang model
import os

# Your model path
MODEL_PATH = r"CC:\Users\Bxlaz\Desktop\Github Repository\Train-Platform-Safety-Breach-Capstone-Project\assets\model\best.pt"

# Verify the file exists
if os.path.exists(MODEL_PATH):
    print(f"✓ Model found at: {MODEL_PATH}")
    print(f"File size: {os.path.getsize(MODEL_PATH) / 1024 / 1024:.2f} MB")
else:
    print(f"✗ Model not found at: {MODEL_PATH}")
    print("Please check the path and try again")

✗ Model not found at: CC:\Users\Bxlaz\Desktop\Github Repository\Train-Platform-Safety-Breach-Capstone-Project\assets\model\best.pt
Please check the path and try again


In [9]:
#FLASK SETUP WITH DETECT.PY RUN_DETECT LOGIC
#Run this cell before startin the server.
import os
import sys
import torch
from flask_cors import CORS
from flask import Flask, request, jsonify
from ultralytics import YOLO
import cv2
import numpy as np
from datetime import datetime
import traceback
from shapely.geometry import Polygon, Point
from collections import defaultdict
import importlib.util
import csv
from datetime import datetime

CSV_LOG_PATH = r"C:\Users\Bxlaz\Desktop\Github Repository\Train-Platform-Safety-Breach-Capstone-Project\Breach_Logs_CSV\breach_logs.csv" # Path to save CSV logs
os.makedirs(os.path.dirname(CSV_LOG_PATH), exist_ok=True)

# Initialize CSV with headers at start of video session
with open(CSV_LOG_PATH, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Timestamp', 'Frame Time (s)', 'Breach Type', 'Track ID', 'X', 'Y'])

def log_breach_to_csv(breach_type, frame_time, track_id, cx, cy):
    """Log breach event to CSV file"""
    try:
        # Ensure the directory exists
        os.makedirs(os.path.dirname(CSV_LOG_PATH), exist_ok=True)
        
        # Check if file exists, if not create it with headers
        file_exists = os.path.exists(CSV_LOG_PATH)
        
        with open(CSV_LOG_PATH, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            
            # Write headers if file is new
            if not file_exists:
                writer.writerow(['Timestamp', 'Frame Time (s)', 'Breach Type', 'Track ID', 'X', 'Y'])
            
            # Write the breach data
            writer.writerow([
                datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                round(frame_time, 2),  # Round to 2 decimal places
                breach_type,
                track_id,
                int(cx),  # Convert to integer
                int(cy)   # Convert to integer
            ])
            
        print(f"[CSV LOG] Breach logged: {breach_type} at ({cx}, {cy}) for person {track_id} at {frame_time}s")
        
    except Exception as e:
        print(f"[ERROR] Failed to write to CSV: {e}")
        traceback.print_exc()

def save_csv_summary(self):
    """Save a summary of all breaches to CSV"""
    try:
        summary_path = CSV_LOG_PATH.replace('.csv', '_summary.csv')
        
        with open(summary_path, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Person ID', 'Total Danger Breaches', 'Total Warning Breaches', 'First Seen', 'Last Seen'])
            
            for person_id, history in self.person_histories.items():
                writer.writerow([
                    person_id,
                    history["total_breaches"]["danger"],
                    history["total_breaches"]["warning"],
                    history["first_seen"].strftime('%Y-%m-%d %H:%M:%S') if history["first_seen"] else 'N/A',
                    history["last_seen"].strftime('%Y-%m-%d %H:%M:%S') if history["last_seen"] else 'N/A'
                ])
                
        print(f"[CSV] Summary saved to: {summary_path}")
        return summary_path
        
    except Exception as e:
        print(f"[ERROR] Failed to save CSV summary: {e}")
        return None
        
# Add the path where detect.py is located
detect_path = r"C:\Users\Bxlaz\Desktop\Clone\Train-Platform-Safety-Breach-Capstone-Project\detect.py"
if detect_path not in sys.path:
    sys.path.append(detect_path)

# Import from detect.py
from detect import zone_definitions  # Adjust this import based on your actual detect.py structure

sort_file_path = r"C:\Users\Bxlaz\Desktop\Clone\Train-Platform-Safety-Breach-Capstone-Project\sort.py"
spec = importlib.util.spec_from_file_location("sort_tracker", sort_file_path)
sort_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(sort_module)
Sort = sort_module.Sort
KalmanBoxTracker = sort_module.KalmanBoxTracker  # Add this line

# Your model path
MODEL_PATH = r"C:\Users\Bxlaz\Desktop\Clone\Train-Platform-Safety-Breach-Capstone-Project\assets\model\best.pt"

class SubwayDetectionServer:
    def __init__(self, model_path):
        # Reset the global tracker ID counter on server initialization
        KalmanBoxTracker.count = 0
        self.app = Flask(__name__)
        CORS(self.app)
        self.model_path = model_path
        self.model = None
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        # Use zones from detect.py
        self.zone_definitions = zone_definitions
        
        # Add detection history for interpolation
        self.detection_history = defaultdict(lambda: {"last_detection": None, "missed_frames": 0})
        
        # Initialize SORT trackers for each camera
        self.trackers = {
            "cam1": Sort(max_age=60, min_hits=1, iou_threshold=0.25),  # More forgiving
            "cam2": Sort(max_age=60, min_hits=1, iou_threshold=0.25),
            "cam3": Sort(max_age=60, min_hits=1, iou_threshold=0.25)
        }
        
        # Track person histories and breach status
        self.person_histories = defaultdict(lambda: {
            "positions": [],
            "breach_history": [],
            "first_seen": None,
            "last_seen": None,
            "total_breaches": {"warning": 0, "danger": 0}
        })
        
        # Statistics
        self.stats = {
            "total_detections": 0,
            "danger_breaches": 0,
            "warning_breaches": 0,
            "breach_logs": [],
            "unique_people": set()
        }
        
        # Print system info
        print(f"Using device: {self.device}")
        if self.device == 'cuda':
            print(f"GPU: {torch.cuda.get_device_name(0)}")
            print(f"CUDA version: {torch.version.cuda}")
        
        # Load model
        self.load_model()
        
        # Setup routes
        self.setup_routes()
    
    def load_model(self):
        if not os.path.exists(self.model_path):
            raise FileNotFoundError(f"Model not found at: {self.model_path}")
        
        try:
            print(f"Loading model from: {self.model_path}")
            self.model = YOLO(self.model_path)
            self.model.to(self.device)
            
            # Warm up the model
            dummy_img = np.zeros((640, 640, 3), dtype=np.uint8)
            with torch.no_grad():
                _ = self.model(dummy_img)
            
            print(f"✓ Model loaded successfully on {self.device}")
        except Exception as e:
            print(f"✗ Error loading model: {e}")
            traceback.print_exc()
            raise
    
    def check_zone_breach_foot(self, x1, y1, x2, y2, camera_name="cam1"):
        """Check if a person's foot position is in danger or warning zone"""
        zones = self.zone_definitions.get(camera_name, self.zone_definitions["cam1"])
        
        # Calculate foot position (bottom center of bounding box)
        foot_x = int((x1 + x2) / 2)
        foot_y = int(y2)  # Bottom of the bounding box
        
        # Create polygons
        point = Point(foot_x, foot_y)
        danger_polygon = Polygon(zones["danger"])
        warning_polygon = Polygon(zones["warning"])
        
        breach_type = None
        if danger_polygon.contains(point):
            breach_type = "danger"
        elif warning_polygon.contains(point):
            breach_type = "warning"
        
        return breach_type, (foot_x, foot_y)
    
    def smooth_bbox(self, current_bbox, track_id, alpha=0.7):
        """Smooth bounding box movements using exponential moving average"""
        if track_id not in self.person_histories:
            return current_bbox

        history = self.person_histories[track_id]["positions"]
        if len(history) == 0:
            return current_bbox

        last_bbox = history[-1]["bbox"]

        # Apply exponential moving average
        smoothed_bbox = [
            alpha * current_bbox[0] + (1 - alpha) * last_bbox[0],
            alpha * current_bbox[1] + (1 - alpha) * last_bbox[1],
            alpha * current_bbox[2] + (1 - alpha) * last_bbox[2],
            alpha * current_bbox[3] + (1 - alpha) * last_bbox[3]
        ]

        return [int(x) for x in smoothed_bbox]
    
    def predict_next_position(self, track_id, current_bbox):
        """Predict next position based on motion history"""
        if track_id not in self.person_histories:
            return current_bbox

        history = self.person_histories[track_id]["positions"]
        if len(history) < 2:
            return current_bbox

        # Calculate velocity from last 2-3 frames
        recent_positions = history[-3:]
        if len(recent_positions) >= 2:
            # Simple velocity calculation
            dx = recent_positions[-1]["bbox"][0] - recent_positions[-2]["bbox"][0]
            dy = recent_positions[-1]["bbox"][1] - recent_positions[-2]["bbox"][1]

            # Apply velocity to predict next position
            predicted_bbox = [
                current_bbox[0] + dx * 0.5,  # Dampen the prediction
                current_bbox[1] + dy * 0.5,
                current_bbox[2] + dx * 0.5,
                current_bbox[3] + dy * 0.5
            ]
            return predicted_bbox

        return current_bbox
    
    def interpolate_missing_detections(self, current_detections, camera_name):
        """Fill in missing detections using last known positions"""
        interpolated = []

        # Get all active tracks
        if hasattr(self.trackers[camera_name], 'trackers'):
            for tracker in self.trackers[camera_name].trackers:
                track_id = tracker.id + 1

                # Check if this track has a current detection
                has_detection = any(d.get('track_id') == track_id for d in current_detections)

                if not has_detection and tracker.time_since_update < 5:
                    # Use predicted position from Kalman filter
                    predicted_bbox = tracker.get_state()[0]
                    x1, y1, x2, y2 = predicted_bbox

                    # Create interpolated detection
                    interpolated_det = {
                        "x1": int(x1),
                        "y1": int(y1),
                        "x2": int(x2),
                        "y2": int(y2),
                        "confidence": 0.7,  # Lower confidence for interpolated
                        "class": 0,
                        "label": "person",
                        "track_id": track_id,
                        "interpolated": True  # Mark as interpolated
                    }

                    # Check breach for interpolated position
                    breach_type, foot_pos = self.check_zone_breach_foot(x1, y1, x2, y2, camera_name)
                    interpolated_det["breach_type"] = breach_type
                    interpolated_det["foot_position"] = foot_pos

                    interpolated.append(interpolated_det)

        return interpolated
    
    def process_frame(self, image, camera_name="cam1"):
        """Process a single frame with improved tracking"""
        zones = self.zone_definitions.get(camera_name, self.zone_definitions["cam1"])
        zone_polygons = {z: Polygon(pts) for z, pts in zones.items()}

        # Run model prediction
        results = self.model.predict(source=image, conf=0.25, save=False)

        detections = []
        det_array = []  # For SORT format

        for result in results:
            if result.boxes is None:
                continue

            boxes = result.boxes.xyxy.cpu().numpy()
            confidences = result.boxes.conf.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy().astype(int)

            for box, conf, cls in zip(boxes, confidences, classes):
                # Only process person class (class 0)
                if cls != 0:
                    continue

                x1, y1, x2, y2 = box[:4]

                # Add to SORT format [x1,y1,x2,y2,confidence]
                # Boost confidence slightly to help with tracking
                boosted_conf = min(conf * 1.1, 1.0)  # Boost by 10%
                det_array.append([x1, y1, x2, y2, boosted_conf])

        # Convert to numpy array for SORT
        if len(det_array) > 0:
            det_array = np.array(det_array)
        else:
            det_array = np.empty((0, 5))

        # Update SORT tracker
        tracked_objects = self.trackers[camera_name].update(det_array)

        # Process tracked objects
        frame_passengers = 0
        current_time = datetime.now()

        for track in tracked_objects:
            x1, y1, x2, y2, track_id = track.astype(int)

            # Use foot approximation
            foot_x = int((x1 + x2) / 2)
            foot_y = int(y2)
            foot_point = Point(foot_x, foot_y)

            # Check zone breach
            breach_type = None
            if zone_polygons["danger"].contains(foot_point):
                breach_type = "danger"
            elif zone_polygons["warning"].contains(foot_point):
                breach_type = "warning"

            # Update person history
            person = self.person_histories[track_id]
            if person["first_seen"] is None:
                person["first_seen"] = current_time
                self.stats["unique_people"].add(track_id)

            person["last_seen"] = current_time
            person["positions"].append({
                "time": current_time.isoformat(),
                "bbox": [x1, y1, x2, y2],
                "foot_position": [foot_x, foot_y],
                "zone": breach_type or "safe"
            })

            # Keep only last 100 positions to prevent memory issues
            if len(person["positions"]) > 100:
                person["positions"] = person["positions"][-100:]

            # Check if this is a new breach for this person
            if breach_type:
                # Check if person was previously in safe zone (new breach)
                prev_positions = person["positions"][-5:-1] if len(person["positions"]) > 1 else []
                was_safe = all(p["zone"] == "safe" for p in prev_positions[-3:]) if prev_positions else True

                if was_safe:
                   
                    frame_time = 0

                    log_breach_to_csv(breach_type, frame_time, int(track_id), foot_x, foot_y)
                     # This is a new breach
                    person["breach_history"].append({
                        "time": current_time.isoformat(),
                        "type": breach_type,
                        "position": [foot_x, foot_y]
                    })
                    person["total_breaches"][breach_type] += 1

                    # Update global stats
                    if breach_type == "danger":
                        self.stats["danger_breaches"] += 1
                    else:
                        self.stats["warning_breaches"] += 1

                    # Log breach
                    self.stats["breach_logs"].append({
                        "timestamp": current_time.isoformat(),
                        "zone": breach_type,
                        "position": [foot_x, foot_y],
                        "camera": camera_name,
                        "person_id": int(track_id)
                    })

            frame_passengers += 1
            self.stats["total_detections"] += 1

            # Find best matching detection for confidence
            confidence = 0.9  # Default confidence
            if len(det_array) == 0 and len(self.trackers[camera_name].trackers) > 0:
                # Calculate IoU with all detections to find best match
                best_iou = 0
                best_conf = 0.9
                for det in det_array:
                    det_x1, det_y1, det_x2, det_y2, det_conf = det

                    # Calculate IoU
                    xx1 = max(x1, det_x1)
                    yy1 = max(y1, det_y1)
                    xx2 = min(x2, det_x2)
                    yy2 = min(y2, det_y2)

                    w = max(0, xx2 - xx1)
                    h = max(0, yy2 - yy1)

                    if w > 0 and h > 0:
                        intersection = w * h
                        area1 = (x2 - x1) * (y2 - y1)
                        area2 = (det_x2 - det_x1) * (det_y2 - det_y1)
                        union = area1 + area2 - intersection
                        iou = intersection / union if union > 0 else 0

                        if iou > best_iou:
                            best_iou = iou
                            best_conf = float(det_conf)

                confidence = best_conf
                interpolated = self.interpolate_missing_detections(detections, camera_name)
                detections.extend(interpolated)

            # Create detection object with tracking info
            detection = {
                "x1": int(x1),
                "y1": int(y1),
                "x2": int(x2),
                "y2": int(y2),
                "confidence": confidence,
                "class": 0,
                "label": "person",
                "breach_type": breach_type,
                "center": [int((x1 + x2) / 2), int((y1 + y2) / 2)],
                "foot_position": [foot_x, foot_y],
                "track_id": int(track_id),  # Add track ID
                "breach_count": person["total_breaches"],  # Add breach history
                "time_in_zone": len([p for p in person["positions"][-10:] if p["zone"] == breach_type]) if breach_type else 0
            }
            detections.append(detection)

        return detections, frame_passengers
    
    def setup_routes(self):
        @self.app.route('/person/<int:person_id>', methods=['GET'])
        def get_person_history(person_id):
            """Get tracking history for a specific person"""
            if person_id in self.person_histories:
                person = self.person_histories[person_id]
                return jsonify({
                    "success": True,
                    "person_id": person_id,
                    "first_seen": person["first_seen"].isoformat() if person["first_seen"] else None,
                    "last_seen": person["last_seen"].isoformat() if person["last_seen"] else None,
                    "total_breaches": person["total_breaches"],
                    "breach_history": person["breach_history"][-10:],  # Last 10 breaches
                    "current_position": person["positions"][-1] if person["positions"] else None
                })
            else:
                return jsonify({
                    "success": False,
                    "error": f"Person {person_id} not found"
                }), 404
        
        @self.app.route('/reset_trackers', methods=['POST'])
        def reset_trackers():
            """Reset all trackers (useful when switching videos)"""
            try:
                data = request.json or {}
                camera = data.get('camera', 'all')
                reset_counter = data.get('reset_counter', True)

                if reset_counter:
                    KalmanBoxTracker.count = 0  # Reset the global ID counter

                if camera == 'all':
                    for cam in self.trackers:
                        self.trackers[cam] = Sort(max_age=30, min_hits=3, iou_threshold=0.3)
                else:
                    if camera in self.trackers:
                        self.trackers[camera] = Sort(max_age=30, min_hits=3, iou_threshold=0.3)

                # Clear person histories
                self.person_histories.clear()

                # Reset unique people tracking
                self.stats["unique_people"].clear()

                return jsonify({
                    "success": True, 
                    "message": f"Trackers reset for {'all cameras' if camera == 'all' else camera}",
                    "id_counter": KalmanBoxTracker.count
                })
            except Exception as e:
                return jsonify({"success": False, "error": str(e)}), 500
        
        @self.app.route('/health', methods=['GET'])
        def health():
            gpu_info = {}
            if self.device == 'cuda':
                gpu_info = {
                    "gpu_name": torch.cuda.get_device_name(0),
                    "memory_allocated": f"{torch.cuda.memory_allocated(0) / 1024**2:.2f} MB",
                    "memory_reserved": f"{torch.cuda.memory_reserved(0) / 1024**2:.2f} MB",
                    "cuda_version": torch.version.cuda
                }
            
            return jsonify({
                "status": "running",
                "model_loaded": self.model is not None,
                "device": self.device,
                "gpu_info": gpu_info,
                "model_path": self.model_path,
                "timestamp": datetime.now().isoformat()
            })
        
        @self.app.route('/detect', methods=['POST'])
        def detect():
            if self.model is None:
                return jsonify({"success": False, "error": "Model not loaded"}), 500
            
            try:
                file = request.files.get('image')
                camera_name = request.form.get('camera', 'cam1')
                
                if not file:
                    return jsonify({"success": False, "error": "No image provided"}), 400
                
                # Read and decode image
                image_bytes = file.read()
                nparr = np.frombuffer(image_bytes, np.uint8)
                image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                
                if image is None:
                    return jsonify({"success": False, "error": "Invalid image"}), 400
                
                # Process frame using the logic from detect.py
                with torch.cuda.amp.autocast(enabled=(self.device == 'cuda')):
                    with torch.no_grad():
                        detections, passenger_count = self.process_frame(image, camera_name)
                
                return jsonify({
                    "success": True,
                    "detections": detections,
                    "count": len(detections),
                    "device": self.device,
                    "camera": camera_name
                })
                
            except Exception as e:
                print(f"Detection error: {e}")
                traceback.print_exc()
                return jsonify({"success": False, "error": str(e)}), 500
        
        @self.app.route('/zones/<camera_name>', methods=['GET'])
        def get_zones(camera_name):
            """Get zone definitions for a specific camera"""
            zones = self.zone_definitions.get(camera_name)
            if zones:
                return jsonify({
                    "success": True,
                    "camera": camera_name,
                    "zones": zones
                })
            else:
                return jsonify({
                    "success": False,
                    "error": f"No zones defined for camera: {camera_name}"
                }), 404
        
        @self.app.route('/stats', methods=['GET'])
        def get_stats():
            return jsonify({
                "total_passengers": self.stats["total_detections"],
                "breach_counts": {
                    "danger": self.stats["danger_breaches"],
                    "warning": self.stats["warning_breaches"]
                },
                "breach_logs": self.stats["breach_logs"][-10:]  # Return last 10 breach logs
            })
        
        @self.app.route('/stats/reset', methods=['POST'])
        def reset_stats():
            # Reset everything including the ID counter
            KalmanBoxTracker.count = 0  

            # Reset all trackers
            for cam in self.trackers:
                self.trackers[cam] = Sort(max_age=30, min_hits=3, iou_threshold=0.3)

            # Clear all histories and stats
            self.person_histories.clear()
            self.stats = {
                "total_detections": 0,
                "danger_breaches": 0,
                "warning_breaches": 0,
                "breach_logs": [],
                "unique_people": set()
            }

            return jsonify({"success": True, "message": "All statistics and trackers reset"})
        
        @self.app.route('/log_breach', methods=['POST'])
        def log_breach():
            """Log a breach event from the frontend"""
            try:
                data = request.json
                breach_type = data.get('type')
                location = data.get('location')
                timestamp = data.get('timestamp', datetime.now().isoformat())
                
                # Map frontend breach types to our internal types
                if breach_type == 'yellow_line':
                    internal_type = 'warning'
                elif breach_type == 'platform_edge':
                    internal_type = 'danger'
                else:
                    internal_type = breach_type
                
                breach_entry = {
                    "timestamp": timestamp,
                    "zone": internal_type,
                    "location": location,
                    "camera": data.get('camera', 'unknown')
                }
                
                self.stats["breach_logs"].append(breach_entry)
                
                # Keep only last 100 entries to prevent memory issues
                if len(self.stats["breach_logs"]) > 100:
                    self.stats["breach_logs"] = self.stats["breach_logs"][-100:]
                
                return jsonify({"success": True, "message": "Breach logged"})
                
            except Exception as e:
                print(f"Error logging breach: {e}")
                return jsonify({"success": False, "error": str(e)}), 500
        
        @self.app.route('/process_video', methods=['POST'])
        def process_video():
            """Process an entire video file (similar to run_detection in detect.py)"""
            try:
                file = request.files.get('video')
                camera_name = request.form.get('camera', 'cam1')
                save_annotated = request.form.get('save_annotated', 'true').lower() == 'true'
                
                if not file:
                    return jsonify({"success": False, "error": "No video provided"}), 400
                
                # Save uploaded video temporarily
                temp_video_path = os.path.join('temp', 'uploaded_video.mp4')
                os.makedirs('temp', exist_ok=True)
                file.save(temp_video_path)
                
                # Process video
                result = self.process_video_file(temp_video_path, camera_name, save_annotated)
                
                # Clean up temp file
                os.remove(temp_video_path)
                
                return jsonify({
                    "success": True,
                    "result": result
                })
                
            except Exception as e:
                print(f"Video processing error: {e}")
                traceback.print_exc()
                return jsonify({"success": False, "error": str(e)}), 500
    
    def process_video_file(self, video_path, camera_name="cam1", save_annotated=True):
        """Updated to include SORT tracking"""
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS) or 30
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        # Reset tracker for this camera before processing
        self.trackers[camera_name] = Sort(max_age=30, min_hits=3, iou_threshold=0.3)
        
        zones = self.zone_definitions[camera_name]
        zone_polygons = {z: Polygon(pts) for z, pts in zones.items()}
        
        output_path = None
        if save_annotated:
            output_dir = "annotated"
            os.makedirs(output_dir, exist_ok=True)
            output_path = os.path.join(output_dir, os.path.basename(video_path))
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        frame_id = 0
        breach_log = []
        person_total = 0
        red_total = 0
        yellow_total = 0
        unique_people = set()
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame_id += 1

            frame_time = frame_id / fps
            
            # Process frame with tracking
            detections, passenger_count = self.process_frame(frame, camera_name)
            
            # Draw zones
            for zone, pts in zones.items():
                color = (0, 255, 0) if zone == "safe" else (0, 255, 255) if zone == "warning" else (0, 0, 255)
                pts_array = np.array(pts, np.int32)
                cv2.polylines(frame, [pts_array], isClosed=True, color=color, thickness=2)
            
            # Draw tracked objects
            for det in detections:
                x1, y1, x2, y2 = det["x1"], det["y1"], det["x2"], det["y2"]
                track_id = det["track_id"]
                breach_type = det["breach_type"]
                foot_x, foot_y = det["foot_position"]

                unique_people.add(track_id)
                
                # Set color based on breach type
                color = (0, 255, 0)  # Green for safe
                label = f"Person #{track_id}"
                
                if breach_type == "danger":
                    color = (0, 0, 255)  # Red
                    label += " - DANGER"
                    log_breach_to_csv(breach_type, frame_time, int(track_id), foot_x, foot_y)
                elif breach_type == "warning":
                    color = (0, 255, 255)  # Yellow
                    label += " - WARNING"
                    log_breach_to_csv(breach_type, frame_time, int(track_id), foot_x, foot_y)
                
                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                
                # Draw label with track ID
                label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
                cv2.rectangle(frame, (x1, y1 - label_size[1] - 4), 
                             (x1 + label_size[0], y1), color, -1)
                cv2.putText(frame, label, (x1, y1 - 2), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                
                # Draw foot position
                foot_x, foot_y = det["foot_position"]
                cv2.circle(frame, (foot_x, foot_y), 5, color, -1)
                cv2.line(frame, (foot_x - 10, foot_y), (foot_x + 10, foot_y), color, 2)
            
            person_total = len(unique_people)
            
            if save_annotated:
                out.write(frame)
        
        cap.release()
        if save_annotated:
            out.release()
        
        # Calculate breach totals from person histories
        for person_id, history in self.person_histories.items():
            red_total += history["total_breaches"]["danger"]
            yellow_total += history["total_breaches"]["warning"]
        
        return {
            "total_passengers": person_total,
            "unique_passengers": len(unique_people),
            "breach_counts": {
                "danger": red_total,
                "warning": yellow_total
            },
            "breaches": breach_log,
            "annotated_video": output_path if save_annotated else None
        }
    
    def run(self, **kwargs):
        """Run the Flask server"""
        self.app.run(**kwargs)

In [11]:
# Start the server with your model
if os.path.exists(MODEL_PATH):
    try:
        # Create and run the server
        server = SubwayDetectionServer(MODEL_PATH)
        
        print("\n" + "="*50)
        print("🚀 Subway Detection Server is running!")
        print("="*50)
        print(f"✓ Model: best.pt")
        print(f"✓ Device: {server.device}")
        if server.device == 'cuda':
            print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
        print(f"\nEndpoints:")
        print(f"  • Health check: http://localhost:5000/health")
        print(f"  • Detection: http://localhost:5000/detect")
        print(f"  • Statistics: http://localhost:5000/stats")
        print("\n⚠️  Press 'Interrupt' button or Kernel → Interrupt to stop")
        print("="*50 + "\n")
        
        # Run the server
        server.run(debug=True, host='0.0.0.0', port=5000, use_reloader=False)
        
    except KeyboardInterrupt:
        print("\n\n✓ Server stopped by user")
    except Exception as e:
        print(f"❌ Error starting server: {e}")
        import traceback
        traceback.print_exc()
else:
    print(f"❌ Cannot start server - model file not found at: {MODEL_PATH}")

Using device: cpu
Loading model from: C:\Users\Bxlaz\Desktop\Clone\Train-Platform-Safety-Breach-Capstone-Project\assets\model\best.pt

0: 640x640 (no detections), 150.4ms
Speed: 3.0ms preprocess, 150.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)
✓ Model loaded successfully on cpu

🚀 Subway Detection Server is running!
✓ Model: best.pt
✓ Device: cpu

Endpoints:
  • Health check: http://localhost:5000/health
  • Detection: http://localhost:5000/detect
  • Statistics: http://localhost:5000/stats

⚠️  Press 'Interrupt' button or Kernel → Interrupt to stop

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.8:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Jul/2025 17:53:49] "GET /health HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2025 17:53:51] "OPTIONS /reset_trackers HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2025 17:53:51] "POST /reset_trackers HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2025 17:53:51] "OPTIONS /reset_trackers HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2025 17:53:51] "POST /reset_trackers HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2025 17:53:56] "GET /stats HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2025 17:54:01] "GET /stats HTTP/1.1" 200 -



0: 416x640 15 persons, 1 train, 105.9ms
Speed: 1.4ms preprocess, 105.9ms inference, 0.9ms postprocess per image at shape (1, 3, 416, 640)


C:\Users\Bxlaz\AppData\Local\Temp\ipykernel_13920\1396441222.py:544: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(self.device == 'cuda')):
127.0.0.1 - - [19/Jul/2025 17:54:03] "POST /detect HTTP/1.1" 200 -



0: 416x640 19 persons, 1 train, 97.3ms
Speed: 2.2ms preprocess, 97.3ms inference, 1.5ms postprocess per image at shape (1, 3, 416, 640)


...

127.0.0.1 - - [19/Jul/2025 17:55:10] "POST /detect HTTP/1.1" 200 -



0: 384x640 3 persons, 88.4ms
Speed: 1.3ms preprocess, 88.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)


127.0.0.1 - - [19/Jul/2025 17:55:10] "POST /detect HTTP/1.1" 200 -



0: 384x640 3 persons, 90.4ms
Speed: 1.4ms preprocess, 90.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)


127.0.0.1 - - [19/Jul/2025 17:55:10] "POST /detect HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2025 17:55:10] "GET /stats HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2025 17:55:10] "OPTIONS /reset_trackers HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2025 17:55:10] "POST /reset_trackers HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2025 17:55:10] "GET /stats HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2025 17:55:11] "GET /stats HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2025 17:55:11] "GET /stats HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2025 17:55:11] "GET /stats HTTP/1.1" 200 -
